<h1> lb1 simple RS

# Init

### imports

In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

### other

In [28]:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']

users = pd.read_csv('movielens/u.user', sep='|', names=u_cols, encoding='latin-1')
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [29]:
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDb URL',
'unknown', 'Action', 'Adventure','Animation', 'Children\'s', 'Comedy', 'Crime', 
'Documentary', 'Drama', 'Fantasy','Film-Noir', 'Horror', 'Musical', 'Mystery',
'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

movies = pd.read_csv('movielens/u.item', sep='|', names=i_cols, encoding='latin-1')
movies.head()

,movie_id,title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [30]:
movies = movies[['movie_id','title']]

In [31]:
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']

ratings = pd.read_csv('movielens/u.data', sep='\t', names=r_cols, encoding='latin-1')
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [32]:
ratings = ratings.drop('timestamp', axis=1)

##  Навчання та тестування

In [33]:
# Призначаємо X як оригінальний датафрейм рейтингів, а y як стовпець user_id ratings.
X = ratings.copy()
y = ratings['user_id']

# Розбиваємо на тренувальний та тестовий набори даних, розшаровані за user_id
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25,
    stratify=y, random_state=42)


In [34]:
def rmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    return np.sqrt(mse)

In [35]:
def baseline(user_id, movie_id):
    return 3.0

In [36]:
def score(cf_model):
    # Створити список пар користувач-фільм із тестового набору даних
    id_pairs = zip(X_test['user_id'], X_test['movie_id'])
    # Передбачити рейтинг для кожної пари користувач-фільм
    y_pred = np.array([cf_model(user, movie) for (user, movie) in id_pairs])
    # Витягти фактичні рейтинги, дані користувачами в тестовому наборі даних
    y_true = np.array(X_test['rating'])
    # Повернути остаточний бал RMSE
    return rmse(y_true, y_pred)

In [37]:
score(baseline)

1.2488234462885457

# Спільна фільтрація на основі користувачів

In [38]:
#Build the ratings matrix using pivot_table function
r_matrix = X_train.pivot_table(values='rating', index='user_id', columns='movie_id')
r_matrix.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1671,1672,1673,1674,1676,1677,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,NaN,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Середнє значення

In [39]:
# Колаборативний фільтр на основі користувачів із середнім рейтингом
def cf_user_mean(user_id, movie_id):
    # Перевірте, чи існує movie_id у матриці r
    if movie_id in r_matrix:
        # Обчисліть середнє значення всіх рейтингів, наданих фільму
        mean_rating = r_matrix[movie_id].mean()
    else:
        # За замовчуванням використовуйте рейтинг 3,0 за відсутності будь-якої інформації
        mean_rating = 3.0
    return mean_rating

# Обчислення RMSE для моделі cf_user_mean
score(cf_user_mean)


1.0300824802393536

## Середньозважене значення

In [40]:
# Створення двійкової матриці рейтингів із усіма нульовими значеннями, що замінюються на 0
r_matrix_dummy = r_matrix.copy().fillna(0)

# Імпорт косинусної оцінки
from sklearn.metrics.pairwise import cosine_similarity

# Обчислення матриці косинусної подібності за допомогою двійкової матриці рейтингів
cosine_sim = cosine_similarity(r_matrix_dummy, r_matrix_dummy)

# Перетворення на pandas dataframe
cosine_sim = pd.DataFrame(cosine_sim, index=r_matrix.index, columns=r_matrix.index)

# Відображення перших 10 рядків
cosine_sim.head(10)


user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.108361,0.046638,0.029577,0.245753,0.335853,0.344724,0.191582,0.057149,0.251979,...,0.257073,0.069412,0.231643,0.108093,0.176842,0.104799,0.232472,0.051528,0.129555,0.256333
2,0.108361,1.000000,0.057613,0.130237,0.054918,0.190552,0.079399,0.076146,0.167992,0.147376,...,0.136993,0.252887,0.255454,0.285193,0.232751,0.149088,0.102807,0.062386,0.109143,0.107686
3,0.046638,0.057613,1.000000,0.139805,0.000000,0.032485,0.043869,0.080968,0.022263,0.059925,...,0.027402,0.000000,0.175060,0.010343,0.105635,0.019052,0.127099,0.023917,0.060392,0.000000
4,0.029577,0.130237,0.139805,1.000000,0.000000,0.045190,0.088586,0.199526,0.135013,0.026919,...,0.055392,0.049773,0.076549,0.139382,0.113886,0.000000,0.130343,0.077357,0.157890,0.063911
5,0.245753,0.054918,0.000000,0.000000,1.000000,0.176443,0.281860,0.132205,0.038790,0.134200,...,0.183969,0.019305,0.073714,0.041807,0.081088,0.029743,0.188392,0.068342,0.055557,0.207259
6,0.335853,0.190552,0.032485,0.045190,0.176443,1.000000,0.394725,0.143385,0.125126,0.372679,...,0.328643,0.070809,0.135806,0.171670,0.125446,0.086464,0.230566,0.095478,0.197307,0.185268
7,0.344724,0.079399,0.043869,0.088586,0.281860,0.394725,1.000000,0.215861,0.121224,0.378723,...,0.339853,0.110866,0.096055,0.104690,0.126108,0.075012,0.270071,0.020036,0.236086,0.266571
8,0.191582,0.076146,0.080968,0.199526,0.132205,0.143385,0.215861,1.000000,0.116173,0.169088,...,0.150048,0.064242,0.118297,0.053969,0.168057,0.095736,0.164157,0.076269,0.089871,0.210995
9,0.057149,0.167992,0.022263,0.135013,0.038790,0.125126,0.121224,0.116173,1.000000,0.152694,...,0.082819,0.064400,0.127051,0.069251,0.095673,0.000000,0.131458,0.106763,0.089297,0.089583


In [41]:
# Колаборативний фільтр на основі користувачів із середнім рейтингом
def cf_user_wmean(user_id, movie_id):
    # За замовчуванням рейтинг 3.0 за відсутності будь-якої інформації
    wmean_rating = 3.0

    # Перевірте, чи існує movie_id у матриці r
    if movie_id in r_matrix:

        # Отримайте бали подібності для користувача, що цікавить, з кожним іншим користувачем
        sim_scores = cosine_sim[user_id]

        # Отримайте рейтинги користувачів для фільму, що цікавить
        m_ratings = r_matrix[movie_id]

        # Витягніть індекси, що містять NaN у ряді m_ratings
        idx = m_ratings[m_ratings.isnull()].index

        # Видаліть значення NaN з m_ratings Series
        m_ratings = m_ratings.dropna()

        # Видаліть відповідні бали косинуса з ряду sim_scores
        sim_scores = sim_scores.drop(idx)

        # Обчисліть остаточний зважений середній
        if len(sim_scores) >= 2:
            wmean_rating = np.dot(sim_scores, m_ratings) / sim_scores.sum()

    return wmean_rating

# Оцініть RMSE для моделі cf_user_wmean
score(cf_user_wmean)


1.022900177976208